<div style="text-align: center; background-color: #750E21; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  PROJECT: BUILDING AND MINING DATA WAREHOUSE 📌
</div>

<div style="text-align: center; background-color: #0766AD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
DATA MINING 📌
</div>

In [9]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

### 📝 **Đọc dữ liệu từ Data Warehouse**

Đầu tiên, ta sẽ đọc dữ liệu từ Data warehouse (AirPollution_DDS). Sau đó chúng ta sẽ kiểm tra kiểu dữ liệu của các cột và xem một vài dòng dữ liệu đầu tiên.

In [10]:
df = pd.read_csv('fact_aqi.csv')

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128430 entries, 0 to 128429
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   full_date       128430 non-null  object
 1   county_name     128430 non-null  object
 2   site_code_DD    128430 non-null  int64 
 3   parameter_name  128430 non-null  object
 4   category_name   128430 non-null  object
 5   aqi             128430 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 5.9+ MB
None
    full_date  county_name  site_code_DD parameter_name category_name  aqi
0  2021-01-01     Macoupin             2            NO2          Good    3
1  2021-01-01  North Slope          9000          PM2.5          Good    3
2  2021-01-01     Harrison             5             CO          Good    3
3  2021-01-01       Wilson          1038            NO2          Good    4
4  2021-01-01      Kleberg           314          PM2.5          Good    5


Tiếp theo, chúng ta sẽ thêm một số cột dữ liệu Input liên quan đến ngày tháng để hỗ trợ tốt hơn cho việc dự đoán kết quả AQI.

In [11]:
# Convert full_date to datetime and add some date features
df['full_date'] = pd.to_datetime(df['full_date'])
df['month'] = df['full_date'].dt.month
df['day'] = df['full_date'].dt.day
df['day_of_week'] = df['full_date'].dt.dayofweek  # 0 = Monday, 6 = Sunday
df['is_weekend'] = df['day_of_week'] >= 5

Sau đó, chúng ta sẽ thực hiện Label Encoder cho cột `county_name`, chuẩn hoá các cột dữ liệu còn lại để hỗ trợ tốt hơn cho việc xây dựng mô hình dự đoán.

In [12]:
# Encode categorical variables
le = preprocessing.LabelEncoder()
df['county_name'] = le.fit_transform(df['county_name'])

scaler = MinMaxScaler()
df['full_date'] = scaler.fit_transform(df[['full_date']])
df['month'] = scaler.fit_transform(df[['month']])
df['day'] = scaler.fit_transform(df[['day']])
df['day_of_week'] = scaler.fit_transform(df[['day_of_week']])
df['is_weekend'] = scaler.fit_transform(df[['is_weekend']])

Bây giờ, chúng ta sẽ lọc ra các thuộc tính Input (X) và thuộc tính Target (y). Và chúng ta cũng sẽ chia bộ dữ liệu theo tỉ lệ 8/2 lần lượt cho tập dữ liệu để Train và tập dữ liệu để Test.

In [13]:
X = df.drop(columns=['full_date', 'aqi', 'site_code_DD', 'category_name', 'parameter_name']) 
y = df['aqi'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Sau khi có các Input và có các tập dữ liệu để Train, ta sẽ tiếp tục xây dựng mô hình dự đoán với thuật toán Random Forest.

In [14]:
# Khởi tạo và huấn luyện mô hình Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_test)

Sau khi đã xây dựng mô hình, chúng ta sẽ kiểm tra các giá trị `MSE`, `MAE`, `R²` và độ chính xác của mô hình.

In [15]:
# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
accuracy = rf_model.score(X_test, y_test)
accuracy = accuracy * 100

# format with 4 decimal places
mse = "{:.4f}".format(mse)
mae = "{:.4f}".format(mae)
r2 = "{:.4f}".format(r2)
accuracy = "{:.4f}".format(accuracy)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")
print(f"Accuracy: {accuracy}%")

Mean Squared Error (MSE): 371.1023
Mean Absolute Error (MAE): 13.0188
R² Score: 0.3147
Accuracy: 31.4654%


- Giá trị MSE khá cao, tuy nhiên trong trường hợp aqi dao động từ 0 - 500 nên vẫn còn có thể chấp nhận được.

- MAE ~ 13, có nghĩa là giá trị dự đoán AQI sai lệch khoảng 13 đơn vị AQI so với giá trị thực.

- R² = 0.3153 là khá thấp. Điều này có nghĩa là mô hình chưa học được tốt mối quan hệ giữa các đặc trưng và giá trị AQI.

- Độ chính xác chỉ là 31.46% nên đây chưa phải là mô hình dự đoán quá tốt, hoặc có thể là chưa đủ dữ liệu Input để thực hiện dự đoán cho mô hình.

Cuối cùng, chúng ta sẽ thực hiện dự đoán cho một `county` với một ngày nào đó trong tương lai.

In [18]:
predict_date = '2024-09-24'
county_name = 'Cochise'

future_data = pd.DataFrame({
    'full_date': pd.to_datetime([predict_date]),  # Future date
    'county_name': [county_name],  # County name
})

# Preprocess the future data
future_data['month'] = future_data['full_date'].dt.month
future_data['day'] = future_data['full_date'].dt.day
future_data['day_of_week'] = future_data['full_date'].dt.dayofweek
future_data['is_weekend'] = future_data['day_of_week'] >= 5
future_data['county_name'] = le.transform(future_data['county_name'])

# Drop unused columns to match the feature set
future_data_preprocessed = future_data.drop(columns=['full_date'])

# Predict AQI
predicted_aqi = rf_model.predict(future_data_preprocessed)
print(f"Predicted AQI: {predicted_aqi}")


Predicted AQI: [41.68]
